In [1]:
import pandas as pd
import numpy as np
import random
import re



In [2]:
df = pd.read_csv('SOLUSTDAtas_tratado.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36400 entries, 0 to 36399
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    36400 non-null  object 
 1   open    36400 non-null  float64
 2   high    36400 non-null  float64
 3   low     36400 non-null  float64
 4   close   36400 non-null  float64
dtypes: float64(4), object(1)
memory usage: 1.4+ MB
None


# Preprocesado de Datos

In [3]:
tamanio = df.shape[0]

In [4]:
NUMHORAS = 7

In [5]:
df_train = df.copy().loc[0:int(tamanio*0.7)]
df_vali = df.copy().loc[int(tamanio*0.7 + 1):int(tamanio*0.9)]
df_test = df.copy().loc[int(tamanio*0.9 + 1):tamanio]

# Desarrollo del modelo

In [ ]:
class RegresionSimbolica:
    def __init__(self, funcionOptimizacion: callable, operations: list[list], maxSize: int, minSize: int, n : int): # n es la cantidad de horas anteriores que se tienen en cuenta
        self.funcionOptimizacion = funcionOptimizacion
        self.operations = operations  # La primera lista contendrá las operaciones más prioritarias y así progresivamente
        self.maxSize = maxSize
        self.minSize = minSize
        self.genes: list[list] = []
        self.n = n
        pass
    
    
    def __create_priv(self, r : int, numGenes: int):
        genes = []
        for j in range (numGenes):   # Se genera un número aleatorio de genes entre 2 y numGenes
            aux = []
            aux.append(random.randint(0, r))
            for i in range(random.randint(self.minSize, self.maxSize - 1)):
                numLO = random.randint(0, len(self.operations) - 1)
                aux.append(self.operations[numLO][random.randint(0, len(self.operations[numLO]) - 1)])
                aux.append(random.randint(0, r))
            genes.append(aux)
        return genes
            
    
    
    def create(self, numGenes: int):
        if numGenes > 0:
            return self.__create_priv(4*self.n - 1, numGenes)
        else:
            raise Exception("El número de genes no puede ser menor que 1")
        
    def __aplica_operacion(self, op, i, j):
        if op == '+':
            return i + j
        elif op == '-':
            return i - j
        elif op == '*':
            return i * j
        elif op == '/':
            return i / j if j != 0 else 1e6
        elif op == '^':
            return i ** j
        
    def __evaluate(self, valores: list[float], candidato: list):
        candidato_aux = []
        for i in range(0, len(candidato) - 1, 2):
            candidato_aux.append(valores[candidato[i]])
            candidato_aux.append(candidato[i+1])
        candidato_aux.append(valores[candidato[len(candidato) - 1]])
        for categoria_operaciones in self.operations:
            j_offset = 0
            for j in range(1, len(candidato_aux), 2):
                indice = j + j_offset
                op = candidato_aux[indice]
                if(op in categoria_operaciones):
                    op_result = self.__aplica_operacion( op, candidato_aux[indice-1], candidato_aux[indice+1])
                    candidato_aux[indice-1] = op_result
                    del candidato_aux[indice+1]
                    del candidato_aux[indice]
                    j_offset = j_offset - 2
        return candidato_aux[0]   
    
    
    def fitness(self, X, y, candidato: list):
        valores_generados = []
        for elem in X:
            valores_generados.append(self.__evaluate(elem, candidato))
        return self.funcionOptimizacion(valores_generados, y)
    
    def fitness2(self, X, y, candidato: list):
        valores_generados = []
        for elem in X:
            valores_generados.append(self.__evaluate(elem, candidato))
        return self.funcionOptimizacion(valores_generados, y)
               
                
        
            
        
    def display(self,  candidato: list):
        return  ' '.join(map(str, [i for i in candidato]))
        
    def mutate(self, candidato: list):
        numberC = (1 + len(candidato)) / 2
        if numberC < self.maxSize and random.randint(0,100) == 0:
            op_cat = random.randint(0, len(self.operations) - 1)
            candidato.append(self.operations[op_cat][random.randint(0, len(self.operations[op_cat]) - 1)])
            candidato.append(random.randrange(0, 4*self.n))
            return candidato
        elif numberC > self.minSize and random.randint(0,50) == 0:
            indice = random.randrange(0, len(candidato) - 1)
            del candidato[indice]
            del candidato[indice]
            return candidato
        return candidato
    
    def mutate2(self, candidato: list):
        numberC = (1 + len(candidato)) / 2
        #for i in range(0,20):
        if numberC < self.maxSize and random.randint(0,50) == 0:
            op_cat = random.randint(0, len(self.operations) - 1)
            candidato.append(self.operations[op_cat][random.randint(0, len(self.operations[op_cat]) - 1)])
            candidato.append(random.randrange(0, 4*self.n))
            numberC += 2
        elif numberC > self.minSize and random.randint(0,50) == 0:
            indice = random.randrange(0, len(candidato) - 1)
            del candidato[indice]
            del candidato[indice]
            numberC -= 2
        return candidato

    def cargar(self, linea):
        gen = []
        gen = re.findall(r'\d+|[+\-*/]', linea)
        gen = [float(tok) if tok.isdigit() else tok for tok in gen]
        return gen



    def runcopy(self, numGenes, X, y, baremo: float, cargar: bool):
        resultado = []
        num_veces = 0
        best = 1000000
        candidato_best = []
        self.genes = []
        if cargar:
            with open('estado.txt', 'r') as archivo:
                for linea in archivo:
                    linea = linea.strip()
                    self.genes.append(self.cargar(linea))
                numGenes = len(self.genes)
        else:
            self.genes = self.create(numGenes)
        for i in range (0, len(self.genes)):
            value = self.fitness2(X, y, self.genes[i])
            if(value < best):
                best = value
                candidato_best = self.genes[i]
        while(best > baremo and num_veces < 1000000):
            for j in range (0, len(self.genes)):
                self.genes[j] = self.mutate2(self.genes[j])
            values_list = []
            for r in range (0, len(self.genes)):
                value = self.fitness2(X, y, self.genes[r])
                values_list.append(value)
                if(value < best):
                    best = value
                    candidato_best = self.genes[r]
            if num_veces > 200 and num_veces % 100 == 0:
                indexed_fitness = list(enumerate(values_list))
                indexed_fitness.sort(key=lambda x: x[1])  # Ordenar por fitness

                mejor_indice = indexed_fitness[0][0]
                aux = self.genes[mejor_indice]

                # Obtener los índices de los 19 peores
                peores_indices = [idx for idx, _ in indexed_fitness[-5:]]
                peores_indices.sort(reverse=True)

                for indice in peores_indices:
                    del self.genes[indice]

                for _ in range(5):
                    self.genes.append(aux)
            num_veces += 1
            print(f"Vez num:{num_veces}, valor{best}")
            genBest = self.display(candidato_best)
            resultado.append({'iteracion' : num_veces, 'valor' : best, 'gen' : genBest})
            with open('genesIteracion.txt', 'a') as archivo:
                archivo.write(f"Vez num:{num_veces}, valor{best}, gen: {genBest} \n")
            if num_veces % 100 == 0:
                df_resultados = pd.DataFrame(resultado)
                cadena = "Dataframes/resultados_regresionSimbolicaC_it" + str(num_veces) + ".csv"
                df_resultados.to_csv(cadena, index=False)
                with open('estado.txt', 'w') as archivo_estado:
                    for gen in self.genes:
                        cadena  = str(self.display(gen)) + "\n"
                        archivo_estado.write(cadena)
        print("El mejor gen tiene un rendimiento de " + str(best))
        self.display(candidato_best)
        df_resultados = pd.DataFrame(resultado)
        cadena = "Dataframes/resultados_regresionSimbolica.csv"
        df_resultados.to_csv(cadena, index=False)
        return (candidato_best, best)


In [30]:
def funcion_optimizacion_mape(valores_generados, y):
    suma = 0
    n = len(valores_generados)
    for i in range(0,n):
        suma = abs(valores_generados[i] - y[i])/y[i] +  suma
    error_medio = suma/n
    return error_medio*100 

In [31]:

operations = [
    ['*', '/'],   # Primera lista de operaciones prioritarias
    ['+', '-']    # Segunda lista de operaciones con menor prioridad
]

# Declarar el objeto
objeto_regresion = RegresionSimbolica(
    funcionOptimizacion=funcion_optimizacion_mape,  # Pasas tu función de optimización
    operations=operations,                          # Pasas la lista de operaciones
    maxSize=50,                                     # Tamaño máximo del cromosoma
    minSize=5,                                      # Tamaño mínimo del cromosoma
    n=NUMHORAS                                            # Cantidad de horas anteriores a considerar
)

In [9]:
print(objeto_regresion.display([1, '+', 32, '*', 5, '-', 6, '+', 9, '/', 2]))
objeto_regresion.fitness([[1, 3,1,4,5,2,5,8,5]], [10.2], [1, '+', 3, '*', 5])
candidato = [1, '+', 3, '*', 5]
for i in range(0, 300):
    objeto_regresion.mutate(candidato)
objeto_regresion.display(candidato)    



1 + 32 * 5 - 6 + 9 / 2


'1 + 3 * 5 * 3'

Preparar X e y

In [10]:
X = []
y= []
for i in range(0, df_train.shape[0] - NUMHORAS):
    aux = []
    for r in range(0, NUMHORAS):
        aux.append(df_train.iloc[i + r].open)
        aux.append(df_train.iloc[i + r].high)
        aux.append(df_train.iloc[i + r].low)
        aux.append(df_train.iloc[i + r].close)
    X.append(aux)
    y.append(df_train.iloc[i + NUMHORAS].close)

In [32]:
pos = objeto_regresion.runcopy(200, X, y, 0.5, False)

Vez num:1, valor210.12764918098816
Vez num:2, valor210.12764918098816
Vez num:3, valor210.12764918098816
Vez num:4, valor210.12764918098816
Vez num:5, valor210.12764918098816
Vez num:6, valor210.12764918098816
Vez num:7, valor210.12764918098816
Vez num:8, valor210.12764918098816
Vez num:9, valor109.83969821083967
Vez num:10, valor109.83969821083967
Vez num:11, valor109.83969821083967
Vez num:12, valor109.83969821083967
Vez num:13, valor109.83969821083967
Vez num:14, valor107.83160893739816
Vez num:15, valor107.83160893739816
Vez num:16, valor107.83160893739816
Vez num:17, valor20.16409617251261
Vez num:18, valor20.16409617251261
Vez num:19, valor20.16409617251261
Vez num:20, valor20.16409617251261
Vez num:21, valor20.16409617251261
Vez num:22, valor20.16409617251261
Vez num:23, valor20.16409617251261
Vez num:24, valor20.16409617251261
Vez num:25, valor20.16409617251261
Vez num:26, valor20.16409617251261
Vez num:27, valor20.16409617251261
Vez num:28, valor20.16409617251261
Vez num:29, v

KeyboardInterrupt: 

In [ ]:
pos

([27, '-', 7, '+', 7], 1.0167578391944412)

In [ ]:
with open('ecuaciones.txt', 'a') as archivo:
    archivo.write(str(pos[0]) + " : " + str(pos[1]) + "_nh" + str(NUMHORAS) + "\n")